In [64]:
import pandas as pd
import re

test = pd.read_csv("2024/A5125600451.csv")
print(test.iloc[0])
print(max(test['MAX']))

STATION                                                   A5125600451
DATE                                                       2024-01-01
LATITUDE                                                      36.6985
LONGITUDE                                                    -93.4022
ELEVATION                                                       411.2
NAME                BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, ...
TEMP                                                             32.7
TEMP_ATTRIBUTES                                                    19
DEWP                                                             24.8
DEWP_ATTRIBUTES                                                    19
SLP                                                            9999.9
SLP_ATTRIBUTES                                                      0
STP                                                             978.4
STP_ATTRIBUTES                                                     19
VISIB               

In [113]:
for year in range(2014, 2025):
    df = pd.read_csv(str(year)+'-Wind.csv')
    ca_df = df[df['NAME'].str.endswith('CA US', na=False)]
    ca_df.to_csv('CA-' + str(year)+ '-Wind.csv', index=False)

/var/folders/0h/y7x46tzs4n7_bwqfd9b2c0q00000gn/T/ipykernel_75761/1758563562.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(year)+'-Wind.csv')
/var/folders/0h/y7x46tzs4n7_bwqfd9b2c0q00000gn/T/ipykernel_75761/1758563562.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(year)+'-Wind.csv')
/var/folders/0h/y7x46tzs4n7_bwqfd9b2c0q00000gn/T/ipykernel_75761/1758563562.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(year)+'-Wind.csv')
/var/folders/0h/y7x46tzs4n7_bwqfd9b2c0q00000gn/T/ipykernel_75761/1758563562.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(str(year)+'-Wind.csv')
/var/folders/0h/y7x46tzs4n7_bwqfd9b2c0q00000gn/T/ipykernel_75761/1758563562.py:2: DtypeWarning: Columns 

In [17]:
import glob
import os

In [18]:
all_files = glob.glob(os.path.join('2024', "*.csv"))

# Read each CSV file and concatenate them into a single DataFrame
df_list = [pd.read_csv(file) for file in all_files]
combined_df = pd.concat(df_list, ignore_index=True)
print(len(df_list))

11875


In [ ]:
import glob
import os
import pandas as pd
def combine_files(year):
    all_files = glob.glob(os.path.join(year, "*.csv"))

    # Read each CSV file and concatenate them into a single DataFrame
    df_list = [pd.read_csv(file) for file in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df.to_csv(year + 'Wind.csv', index=False)
combine_files('2021')

In [108]:
combined_df.to_csv('2024-Wind.csv', index=False)

In [109]:
for year in ['2023', '2022']:
    all_files = glob.glob(os.path.join(year, "*.csv"))

    # Read each CSV file and concatenate them into a single DataFrame
    df_list = [pd.read_csv(file) for file in all_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df.to_csv(year + '-Wind.csv', index=False)



In [56]:
print(len(combined_df))

1833000


In [84]:
# Filter out missing data values of 999.9
combined_df = combined_df[combined_df['WDSP'] != 999.9]

# Change missing values to -1
combined_df['GUST'] = combined_df['GUST'].replace(999.9, -1)
combined_df['MXSPD'] = combined_df['MXSPD'].replace(999.9, -1)
print(len(combined_df))

1768032


IndexError: single positional indexer is out-of-bounds

In [78]:
ca_df = combined_df[combined_df['NAME'].str.contains('CA US', regex=True, na=False)]

In [83]:
import numpy as np
def generate_best_windmill_locations(location):
    df = combined_df[combined_df['NAME'].str.endswith(location, na=False)]
    stations = list(set(df['NAME']))
    wind_df = pd.DataFrame(columns=["STATION", "WDSP", "MXSPD", "GUST"])
    wind_df['STATION'] = stations
    for i in range(len(stations)):
        station = stations[i]
        station_df = df[df['NAME'] == station]
        wind_df.loc[i, 'WDSP'] = np.median(station_df['WDSP'])
        wind_df.loc[i, 'MXSPD'] = np.median(station_df['MXSPD'])
        wind_df.loc[i, 'GUST'] = np.median(station_df['GUST'])
    sorted_wind_speeds = wind_df.sort_values(by='WDSP', ascending=False)
    return sorted_wind_speeds
# print(generate_best_windmill_locations('CA US'))
print(generate_best_windmill_locations('US'))

                                       STATION  WDSP  MXSPD   GUST
1650                   MOUNT WASHINGTON, NH US  28.9   45.1  49.45
662                            TIN CITY, AK US  20.3   28.0   -1.0
2471                       MONARCH PASS, CO US  19.8   34.0   41.0
656               CAPE ROMANZOF AIRPORT, AK US  17.9   28.9   41.0
1620                      BERTHOUD PASS, CO US  17.3   31.1   42.0
...                                        ...   ...    ...    ...
1327  MIDDLE PENINSULA REGIONAL AIRPORT, VA US   0.6    9.9   14.0
6          OAKDALE ALLEN PARISH AIRPORT, LA US   0.5    9.9   -1.0
2511               HUDSON RIVER RESERVE, NY US   0.5    2.9   -1.0
1374           WINDER BARROW CO AIRPORT, GA US   0.2   14.0   -1.0
1822           PAMPA MESA VISTA AIRPORT, TX US   0.0  999.9   -1.0

[2529 rows x 4 columns]


In [39]:
def group_by(df, group_col, agg_func):
    grouped_df = df.groupby(group_col).agg(agg_func).reset_index()
    return grouped_df

Grouping CA DF by Station

In [40]:
agg_func = lambda x: x.mode().iloc[0]
ca_grouped = group_by(ca_df, ["NAME", "LONGITUDE", "LATITUDE", "STATION"], agg_func)
print(ca_grouped.iloc[0])

NAME                ALAMEDA, CA US
LONGITUDE                 -122.298
LATITUDE                    37.772
STATION                99403399999
DATE                    2024-01-01
ELEVATION                      2.0
TEMP                          55.5
TEMP_ATTRIBUTES                 23
DEWP                        9999.9
DEWP_ATTRIBUTES                  0
SLP                         1014.0
SLP_ATTRIBUTES                  23
STP                          999.9
STP_ATTRIBUTES                   0
VISIB                        999.9
VISIB_ATTRIBUTES                 0
WDSP                           6.1
WDSP_ATTRIBUTES                 23
MXSPD                         12.0
GUST                         999.9
MAX                           58.3
MAX_ATTRIBUTES                   *
MIN                           51.6
MIN_ATTRIBUTES                   *
PRCP                           0.0
PRCP_ATTRIBUTES                  I
SNDP                         999.9
FRSHTT                           0
Name: 0, dtype: obje

In [68]:
state_codes = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

In [80]:
# for state in state_codes:
#     best_locs = generate_best_windmill_locations(state + ' US')
#     best_locs.to_csv(state + " Wind.csv", index=False)

In [104]:
print(list(state_codes.keys()).index('MI'))

21


In [88]:
from decimal import Decimal
def convert_to_decimal(item):
    for key, value in item.items():
        if isinstance(value, float):
            item[key] = Decimal(str(value))
        elif isinstance(value, dict):
            convert_to_decimal(value)
    return item

In [102]:
test = generate_best_windmill_locations('AK US')
print(test.iloc[0])
print(len(test))
print(list(set(test['STATION'])))

STATION    TIN CITY, AK US
WDSP                  20.3
MXSPD                 28.0
GUST                  -1.0
Name: 21, dtype: object
172
['ANAKTUVUK AUTO, AK US', 'MEKORYUK, AK US', 'KING SALMON AIRPORT, AK US', 'ILIAMNA AIRPORT, AK US', 'WALES AIRPORT, AK US', 'CHIGNIK AIRPORT, AK US', 'MINCHUMINA AIRPORT, AK US', 'DEADHORSE, AK US', 'DEADHORSE ALPINE AIRSTRIP, AK US', 'VALDEZ MUNICIPAL AIRPORT, AK US', 'UNALAKLEET FIELD, AK US', 'SELAWIK AIRPORT, AK US', 'KIVALINA AIRPORT, AK US', 'KIPNUK AIRPORT, AK US', 'COLD BAY AIRPORT, AK US', 'NELSON LAGOON AIRPORT, AK US', 'CAPE ROMANZOF AIRPORT, AK US', 'PRUDHOE BAY, AK US', 'SAINT PAUL ISLAND AIRPORT, AK US', 'PILOT POINT, AK US', 'ANIAK AIRPORT, AK US', 'TIN CITY, AK US', 'IGIUGIG AIRPORT, AK US', 'EAGLE AIRPORT, AK US', 'UNALASKA, AK US', 'NAPAKIAK, AK US', 'FALSE PASS, AK US', 'COLDFOOT AIRPORT, AK US', 'MARMION ISLAND, AK US', 'SAND POINT AIRPORT, AK US', 'TELLER AIRPORT, AK US', 'KODIAK AIRPORT, AK US', 'YAKUTAT AIRPORT, AK US', 'PETERSB

In [107]:
import boto3

def upload_dynamo_data(table_name, location):
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    best_locs = generate_best_windmill_locations(location)

    # Reference the DynamoDB table
    table = dynamodb.Table(table_name)

    # Example data to be uploaded
    data = []
    for index, row in best_locs.iterrows():
        data.append({'Station': row['STATION'], 'WDSP': row['WDSP'], 'MXSPD': row['MXSPD'], 'GUST': row['GUST']})

    # Convert all float values to Decimal
    data = [convert_to_decimal(item) for item in data]

    # Upload data to DynamoDB table
    for item in data:
        table.put_item(Item=item)

upload_dynamo_data('US-Wind-Data', 'US')

# def get_dynamo_data(table_name, )

In [106]:

# Initialize a session using Amazon DynamoDB
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
i = 0
for state in state_codes:
    print(state)
    if i < 21:
        i += 1
        continue
    best_locs = generate_best_windmill_locations(state + ' US')

    # Reference the DynamoDB table
    table = dynamodb.Table(state + '-Wind-Data')

    # Example data to be uploaded
    data = []
    for index, row in best_locs.iterrows():
        data.append({'Station': row['STATION'], 'WDSP': row['WDSP'], 'MXSPD': row['MXSPD'], 'GUST': row['GUST']})

    # Convert all float values to Decimal
    data = [convert_to_decimal(item) for item in data]

    # Upload data to DynamoDB table
    for item in data:
        table.put_item(Item=item)

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


In [ ]:
table_name = 'AZ-Wind-Data'  # Replace with your table name
table = dynamodb.Table(table_name)

# Check if the table exists
existing_tables = dynamodb.meta.client.list_tables()['TableNames']
if table_name not in existing_tables:
    print(f"Table '{table_name}' does not exist.")

In [91]:
from boto3.dynamodb.conditions import Key

# Initialize a session using Amazon DynamoDB
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

table = dynamodb.Table('CA-Wind-Data')

response = table.get_item(
    Key={
        'Station': 'LANCASTER, CA US'
    }
)

# Print the item retrieved
item = response.get('Item')
if item:
    print("Item retrieved:")
    print(item)
else:
    print("Item not found")

Item retrieved:
{'MXSPD': Decimal('22'), 'WDSP': Decimal('10.8'), 'Station': 'LANCASTER, CA US', 'GUST': Decimal('28.9')}


In [21]:
print(list(set(test['NAME'])))

['BRANSON WEST MUNICIPAL EMERSON FIELD AIRPORT, MO US']


In [9]:
import h5py

# Open the HDF5 file
with h5py.File('OMI-Aura_L2-OMUVB_2020m1006t0308-o86310_v003-2020m1009t080256.he5?A-userid=tgallup&Expires=1719093021&Signature=GPZOY9Atxlam7FrNAxSaoJ6OWPcBHx1-V3fJkL-3eCd3OSJSJrAryoVLp~MwTV97BCld9-o78c6TzyB8dbothC7Pk~I~xEqmc58xxMLONiZiX-k5JinItS1PbDtl.h5', 'r') as f:
    # Access datasets
    dataset = f['HDFEOS/SWATHS/UVB/Geolocation Fields/Longitude']
    print(dataset)
    data = dataset[()]
    print(data)

<HDF5 dataset "Longitude": shape (1644, 60), type "<f4">
[[ -5.329755  -11.240461  -16.333044  ... -84.042366  -85.279755
  -86.6727   ]
 [ -5.1514144 -11.110547  -16.250856  ... -84.32897   -85.558205
  -86.941414 ]
 [ -4.968554  -10.976948  -16.165974  ... -84.61714   -85.838036
  -87.211296 ]
 ...
 [ 29.278193   27.726763   26.332457  ... -50.64947   -57.69378
  -65.80564  ]
 [ 28.967983   27.40267    25.996675  ... -50.51798   -57.463085
  -65.47645  ]
 [ 28.658628   27.079678   25.66222   ... -50.391552  -57.240196
  -65.15714  ]]
